
| ![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-smaller.PNG)  | [National Science Data Fabric](https://nationalsciencedatafabric.org/) [Jupyter notebook](https://jupyter.org/) <br> created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html)  |   ![CEDMAV](https://www.sci.utah.edu/~pascucci/public/cedmav-small.png)  |   ![SCI](https://www.sci.utah.edu/~pascucci/public/SCI-logo-small.png)  |
|---|:---:|:---:|:---:|
| <img src="https://www.sci.utah.edu/~pascucci/public/seal-storage-small.png"  height="50">| 200TB of cloud space provided by [Seal Storage](https://www.sealstorage.io/)  |   ![NASA](https://www.sci.utah.edu/~pascucci/public/NASA_logo_small.png)| Climate data provided by  [NASA](https://gmao.gsfc.nasa.gov/global_mesoscale/dyamond_phaseII/)  |     |


#  <center> This is a preview of the 200TB data visualized interactively below:
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/NSDF-NASA.gif)

# Import Bokeh

In [1]:
import os,sys,logging,time

# import bokeh
os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*" # for debugging 
os.environ["BOKEH_LOG_LEVEL"]="error"   # for debugging 

os.environ["VISUS_USE_PUBLIC_IP"]=str(False)
os.environ["VISUS_SSL_TUNNELS"]=""

import openvisuspy
from openvisuspy import ShowBokehApp, TestBokehApp, Slice, Slices, GetBackend, SetupLogger

logger=SetupLogger(stream=False)
logger.info(f"GetBackend={GetBackend()}")

import bokeh.io
bokeh.io.output_notebook()
ShowBokehApp(TestBokehApp)

Loading BokehJS ...

# Import OpenVisus

In [2]:
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
os.environ["VISSU_CPP_VERBOSE"]="1"

import OpenVisus as  ov
from openvisuspy import Slice,Slices,LoadDataset,ExecuteBoxQuery

print("OpenVisus imported")

OpenVisus imported


# Load Dataset and compute range

In [3]:
import os
access_key="any"
secret_key="any"
endpoint_url="https://maritime.sealstorage.io/api/v0/s3"
key="utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx"
cached="idx"

url="".join([
    f"{endpoint_url}/{key}?",
    f"&access_key={access_key}"
    f"&secret_key={secret_key}"
    f"&endpoint_url={endpoint_url}"
    f"&cached={cached}"
])

print(url)
db=LoadDataset(url)
# print(db.getDatasetBody().toString())
W,H,D=db.getLogicBox()[1]
print("logic box",W,H,D)
access=db.createAccess()

https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx?&access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=idx
logic box 8640 6480 90


In [4]:
import numpy as np

# get a Z slice in the middle to compute the range of the data
quality=-6
endh=db.getMaxResolution()+quality
Z=D//2
logic_box, delta, num_pixels=db.getAlignedBox([[0,0,Z],[W,H,Z]], endh, slice_dir=2)
data=list(ExecuteBoxQuery(db, access=access, logic_box=logic_box,  endh=endh, num_refinements=1))[0]['data']

# guess the palette range from data min-max
palette_range = np.min(data)/4, np.max(data)/4   
print("palette_range",palette_range)

palette_range (-0.25256651639938354, 0.3600933849811554)


In [5]:
def MyApp(doc, timestep=2015, timestep_delta=10, quality=-6):
    
    view=Slices(
        show_options=["palette","timestep","timestep-delta","field","quality","play-button", "play-sec"],
        slice_show_options=["direction","offset","status_bar"])

    view.setNumberOfViews(1)  
    view.setDataset(url)
    view.setPalette("colorcet.coolwarm")
    view.setPaletteRange(palette_range)
    view.setTimestepDelta(timestep_delta)
    view.setTimestep(timestep)
    view.setField(view.getField())
    view.setQuality(quality)
    view.setLogicToPixel([(0.0,1.0), (0.0,1.0), (0.0,30.0)])
    
    view.children[0].setDirection(2)
    view.children[0].setOffset(0)
    
    doc.add_root(view.getBokehLayout(doc=doc))
    
ShowBokehApp(MyApp)